# Pulumi를 사용한 기존 리소스 임포트

이 Notebook은 Pulumi를 사용하여 기존 클라우드 리소스를 임포트하는 방법을 설명한다. Pulumi는 클라우드 인프라를 코드로 관리할 수 있는 도구이다.

## 1. Pulumi 설치

Pulumi는 다양한 설치 방법을 제공한다. 여기서는 curl을 사용한 설치 방법을 설명한다.

In [1]:
# Linux 또는 macOS 시스템에서는 아래 명령어를 사용하여 Pulumi를 설치할 수 있다.
!curl -fsSL https://get.pulumi.com | sh


설치가 완료되면, Pulumi CLI가 시스템 경로에 추가되어야 한다. 만약 자동으로 추가되지 않았다면, 수동으로 추가해야 한다. 예를 들어, `~/.bashrc` 또는 `~/.zshrc` 파일에 다음을 추가한다:

In [2]:
# Pulumi 경로 추가 예제
PATH=$PATH:~/.pulumi/bin
!source ~/.bashrc

## 2. Pulumi 프로젝트 설정

Pulumi는 프로젝트 단위로 인프라를 관리한다. 프로젝트를 설정하려면 `pulumi new` 명령어를 사용한다. 여기서는 Python 템플릿을 사용한다.

In [3]:
# 프로젝트를 위한 새 디렉터리를 생성하고 이동한다.
!mkdir pulumi-import-demo && cd pulumi-import-demo

# Python 프로젝트를 초기화한다.
!pulumi new python -y

Created project 'pulumi-import-demo'
Successfully created virtual environment!
Installing dependencies...
Finished installing dependencies!

Your new project is ready to go! Follow these steps:

    Run `cd pulumi-import-demo`
    Run `pulumi up` to create your stack
    Run `pulumi stack output` to see the output

More info: https://www.pulumi.com/docs/get-started/python/


위 명령은 새로운 Pulumi 프로젝트를 초기화하고 기본 의존성을 설치한다. `Pulumi.yaml` 파일이 생성되고, 기본적인 Python 스크립트가 포함된다.

## 3. 기존 리소스 임포트

다음 단계에서는 기존 AWS EC2 보안 그룹을 임포트하는 방법을 설명한다. `__main__.py` 파일에 다음 코드를 추가하여 보안 그룹을 임포트한다.

In [4]:
%%writefile pulumi-import-demo/__main__.py

import pulumi
import pulumi_aws as aws
from pulumi import ResourceOptions

# 기존 보안 그룹 임포트
group = aws.ec2.SecurityGroup('my-sg',
    name='my-sg-62a569b',
    description='Enable HTTP access',
    ingress=[
        { 'protocol': 'tcp', 'from_port': 80, 'to_port': 80, 'cidr_blocks': ['0.0.0.0/0'] }
    ],
    opts=ResourceOptions(import_='sg-04aeda9a214730248'))


Writing pulumi-import-demo/__main__.py


이 코드는 ID가 `sg-04aeda9a214730248`인 AWS EC2 보안 그룹을 임포트한다.

## 4. Pulumi 스택 배포

이제 `pulumi up` 명령어를 사용하여 리소스를 임포트하고 스택에 추가할 수 있다.

In [5]:
# pulumi 스택 배포 및 리소스 임포트
!cd pulumi-import-demo && pulumi up --yes

Previewing update (dev):
     Type                 Name                       Plan
+   pulumi:pulumi:Stack  pulumi-import-demo-dev     create
=   └─ aws:ec2:SecurityGroup  my-sg                import

Resources:
    + 1 to create
    = 1 to import

Updating (dev):
     Type                 Name                       Status
+   pulumi:pulumi:Stack  pulumi-import-demo-dev     created
=   └─ aws:ec2:SecurityGroup  my-sg                imported

Resources:
    + 1 created
    = 1 imported

Duration: 4s


임포트가 완료되면, Pulumi 프로그램에 임포트된 리소스의 코드를 추가할 수 있다.

## 5. Pulumi 스택 제거

테스트가 완료되면 `pulumi destroy` 명령어를 사용하여 생성된 모든 리소스를 제거할 수 있다.

In [6]:
# pulumi 스택 제거
!cd pulumi-import-demo && pulumi destroy --yes

Previewing destroy (dev):
     Type                 Name                       Plan
-   pulumi:pulumi:Stack  pulumi-import-demo-dev     delete
-   └─ aws:ec2:SecurityGroup  my-sg                delete

Resources:
    - 2 to delete

Destroying (dev):
     Type                 Name                       Status
-   pulumi:pulumi:Stack  pulumi-import-demo-dev     deleted
-   └─ aws:ec2:SecurityGroup  my-sg                deleted

Resources:
    - 2 deleted

Duration: 3s


이 명령은 Pulumi가 모든 리소스를 제거하고 환경을 초기 상태로 되돌린다.

## 6. 정리

이 Notebook에서는 Pulumi를 사용하여 기존 클라우드 리소스를 임포트하고 관리하는 방법을 다루었다. Pulumi를 사용하면 코드로 클라우드 리소스를 쉽게 관리할 수 있다.